In [2]:
import openrouteservice as ors
import folium
from pandas import DataFrame
import pandas as pd
import math
import time
client = ors.Client(key='5b3ce3597851110001cf6248d3629fc815a24f0582ed2410062c812f')


#Please note that due to the daily limit of the website and the limit of the number of reads per minute, it is recommended to read only ten rental data at a time
rent = pd.read_csv("../rent_data/renew(15)")
rent = rent[250:280].reset_index(drop=True)
train = pd.read_excel("../data/raw/train_station_20to21.xlsx")

In [3]:
"""
function: calculate coordinates' distance
param datas : durations, distances
"""
def cal_distance(coordinates, client):
    matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False,
    )
    time.sleep(2.5)
    return matrix['durations'], matrix['distances']
# the unit of distance is m, the unit of durations is seconds

In [4]:
"""
function: The number of times the code needs to run to the one house and all train stations.
param datas : times_i(corresponding i which is in function), times_function
"""
#as api only apply 3500 times each time
def times(len):
    times_i = int(math.sqrt(3500))-1
    times = math.ceil(len/times_i)
    times_function = math.ceil((len+times)/times_i)
    return times_i, times_function

In [5]:
"""
function: Find the nearest site to rent 
param datas : rent_address, stop_name, min_distance, min_duration
"""
def get_coordinates(rent, train):
    rent_address = []
    stop_name = []
    min_distance = []
    min_duration = []
    for i in range(len(rent)):
        coordinates = []
        distances = []
        durations = []
        for j in range(len(train)):
            coordinates.append([train["Stop_long"][j], train["Stop_lat"][j]])
        for k in range(times(len(train))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            matrix = cal_distance(coor_part, client)
            distances = distances + matrix[1][0][1:]
            durations = durations + matrix[0][0][1:]
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        stop_name.append(train['Stop_name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, stop_name, min_distance, min_duration

In [6]:
"""
function: put the required columns into a dataframe
param datas: dataframe
"""
def data_frame(rent, train):
    #get dataframe
    data = {'rent_address': get_coordinates(rent, train)[0], 'stop_name': get_coordinates(rent, train)[1], 'min_distance': get_coordinates(rent, train)[2], 'min_duration': get_coordinates(rent, train)[3]}
    df = DataFrame(data)
    return df

df = data_frame(rent, train)
df
#df.to_csv("/home/ads/generic-real-estate-consulting-project-group-55/data/raw/distances_durations(25)", index=False)

/Users/zhangwenwen/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zhangwenwen/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zhangwenwen/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zhangwenwen/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Use

KeyboardInterrupt: 